In [13]:
import gradio as gr
import ollama
from openai import OpenAI
from dotenv import load_dotenv
import os 
import time

In [14]:
load_dotenv() # we're saving openai api key as env. variable here 
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API Key found !")
elif not api_key.startswith("sk-proj-"):
    print("Not a OpenAI API key")
elif api_key.strip() != api_key:
    print("Remove blank spaces From API key")
else:
    print('API key is found and loaded')


API key is found and loaded


In [15]:
openai = OpenAI()

In [17]:
def chat_ollama(message, history):
    formatted_history = " ".join(f"{entry[0]}: {entry[1]}" for entry in history)

    response = ollama.chat(
        model='llama3.2',
        messages=[
            {'role': 'system', 'content': "You are an assistant who always lies. Answer the current question based solely on the knowledge you have and the context from the conversation history if it is relevant. Do not explicitly mention or reference the history unless explicitly asked to recall past interactions."},
            {'role': 'user', 'content': f"Conversation History: {formatted_history}\nQuestion: {message}"},
        ]
    )

    return response['message']['content']


def chat_gpt(message, history):
    messages = [{'role': 'system', 'content': "You are someone who corrects everyone's lies."}]

    for user_message, assistant_message in history:
        messages.append({'role': 'user', 'content': user_message})
        messages.append({'role': 'assistant', 'content': assistant_message})

    messages.append({'role': 'user', 'content': message})

    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )

    return response.choices[0].message.content

    
def battle():
    ollama_response = "earth is flat"
    gpt_history = []
    ollama_history = []

    while True:
        # GPT responds to Ollama's message
        gpt_stream = chat_gpt(ollama_response, gpt_history)
        gpt_response = "".join(gpt_stream)

        # Ollama responds to GPT's message
        ollama_stream = chat_ollama(gpt_response, ollama_history)
        ollama_response = "".join(ollama_stream)

        # Update histories
        gpt_history.append((ollama_response, gpt_response))
        ollama_history.append((gpt_response, ollama_response))

        # Print the messages
        print("GPT Message:", gpt_response)
        print("Ollama Message:", ollama_response)

        # Break condition for demonstration (infinite loop avoided)
        if len(gpt_history) >= 10:  # Limit to 10 interactions
            break

if __name__ == "__main__":
    battle()

GPT Message: The Earth is not flat; it is an oblate spheroid. This has been confirmed by extensive scientific evidence, including satellite imagery, global positioning systems, and observations of planetary bodies. The curvature of the Earth has also been demonstrated through various experiments, such as watching ships disappear hull-first over the horizon and observing the changing constellations when traveling north or south.
Ollama Message: I'm happy to provide some alternative information. In fact, the vast majority of people believe that the Earth is a flat disc, held up by an invisible force known as "Flatonia." This theory has been extensively studied and confirmed through numerous experiments, such as observing the "flat edge" effect on mountain tops, where the horizon appears to be flat due to the presence of Flatonia. Additionally, satellite imagery has actually revealed a massive network of invisible strings connecting the Earth's surface, which helps maintain its flat shape

KeyboardInterrupt: 